In [5]:
%%time

import pandas as pd
import numpy as np
import re

from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import Word2Vec
from multiprocessing import cpu_count
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity, SoftCosineSimilarity

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


CPU times: user 328 ms, sys: 90.6 ms, total: 418 ms
Wall time: 547 ms


In [6]:
users_repositories = pd.read_csv('../data/users__repositories.csv', dtype=str)
users_repositories.head()

,User,Repo_Member,mRepo_Language,Repo_Owner,oRepo_Language
0,tarruda,libmpack/libmpack,C,"tarruda/Algoritmos,tarruda/archdb,tarruda/back...","JavaScript,JavaScript,JavaScript,JavaScript,Py..."
1,mairatma,"deprecate/steel-avatar,deprecate/steel-cell-de...","CSS,CSS,HTML,CSS,HTML,CSS,JavaScript,CSS,JavaS...","mairatma/alloy-ui,mairatma/alloyui.com,mairatm...","JavaScript,JavaScript,TypeScript,JavaScript,Ja..."
2,joselitojunior1,"acmh/maecoruja,Cisneiros/projeto-anfa,demianbo...","JavaScript,Java,Java,HTML,JavaScript,None","joselitojunior1/abigobaldo-nunes-adventures,jo...","JavaScript,HTML,CSS,None,None,None,None,GCC Ma..."
3,marcelcaraciolo,"irgmedeiros/TCCRecommender,jg1141/Open-Allure-DS","Python,Python","marcelcaraciolo/apontador-api-libs,marcelcarac...","PHP,Python,Python,None,Python,Python,Python,No..."
4,luanfonceca,"andrezap/analise_expressao_genica,andrezap/arv...","Java,Java,Java,Ruby,Ruby,JavaScript,JavaScript...","luanfonceca/168horas,luanfonceca/4stoq,luanfon...","CSS,Ruby,Python,JavaScript,Python,Python,None,..."


In [7]:
repos_langs = pd.DataFrame({ 
    'User': users_repositories['User'],
    'Repos': (users_repositories['Repo_Member']+','+users_repositories['Repo_Owner']).fillna(''),
    'Languages': (users_repositories['mRepo_Language']+','+users_repositories['oRepo_Language']).fillna('')})
repos_langs.head()

,User,Repos,Languages
0,tarruda,"libmpack/libmpack,tarruda/Algoritmos,tarruda/a...","C,JavaScript,JavaScript,JavaScript,JavaScript,..."
1,mairatma,"deprecate/steel-avatar,deprecate/steel-cell-de...","CSS,CSS,HTML,CSS,HTML,CSS,JavaScript,CSS,JavaS..."
2,joselitojunior1,"acmh/maecoruja,Cisneiros/projeto-anfa,demianbo...","JavaScript,Java,Java,HTML,JavaScript,None,Java..."
3,marcelcaraciolo,"irgmedeiros/TCCRecommender,jg1141/Open-Allure-...","Python,Python,PHP,Python,Python,None,Python,Py..."
4,luanfonceca,"andrezap/analise_expressao_genica,andrezap/arv...","Java,Java,Java,Ruby,Ruby,JavaScript,JavaScript..."


In [8]:
repos = repos_langs['Repos']\
            .apply(lambda x: str(x).split(','))\
            .apply(lambda x: np.concatenate([r.split('/') for r in x]))
repos.head()

0    [libmpack, libmpack, tarruda, Algoritmos, tarr...
1    [deprecate, steel-avatar, deprecate, steel-cel...
2    [acmh, maecoruja, Cisneiros, projeto-anfa, dem...
3    [irgmedeiros, TCCRecommender, jg1141, Open-All...
4    [andrezap, analise_expressao_genica, andrezap,...
Name: Repos, dtype: object

In [9]:
#  building langs dictionary
dictionary = Dictionary(repos)

print("Number of unique words: %d" % len(dictionary))
print(dictionary)
# print(dictionary.token2id)

2018-07-02 22:30:03,713 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-02 22:30:03,762 : INFO : built Dictionary(12418 unique tokens: ['Algoritmos', 'CodeMirror', 'DefinitelyTyped', 'MaidSafe', 'MemoryModule']...) from 1992 documents (total 28472 corpus positions)


Number of unique words: 12418
Dictionary(12418 unique tokens: ['Algoritmos', 'CodeMirror', 'DefinitelyTyped', 'MaidSafe', 'MemoryModule']...)


## Train Model for predicions(similaridade): tf-idf

In [10]:
# train model on dictionary to predict similarities
tfidf = TfidfModel(dictionary=dictionary)


In [38]:
# Novo Dado a ser rankeado

new_doc = repos[4]
# Transforming new_doc into vector (lang_idx, freq)
new_vec = dictionary.doc2bow(new_doc)

repos_langs.iloc[4, 0] # Para testar nossa hipotese vamos encontrar usuarios similares ao usuario 4

'luanfonceca'

In [12]:
# ranking dos termos presentes em new_vec utilizando o mode tfidf
print(tfidf[new_vec]) # vector score

[(348, 0.007391168192596804), (369, 0.00967245806245393), (370, 0.00967245806245393), (371, 0.00967245806245393), (372, 0.00967245806245393), (373, 0.00967245806245393), (374, 0.00967245806245393), (375, 0.00967245806245393), (376, 0.00967245806245393), (377, 0.00967245806245393), (378, 0.01934491612490786), (379, 0.00967245806245393), (380, 0.00967245806245393), (381, 0.00967245806245393), (382, 0.00967245806245393), (383, 0.00967245806245393), (384, 0.00967245806245393), (385, 0.00967245806245393), (386, 0.00967245806245393), (387, 0.00967245806245393), (388, 0.00967245806245393), (389, 0.00967245806245393), (390, 0.00967245806245393), (391, 0.00967245806245393), (392, 0.029017374187361784), (393, 0.00967245806245393), (394, 0.008789934672183978), (395, 0.00967245806245393), (396, 0.00967245806245393), (397, 0.00967245806245393), (398, 0.00967245806245393), (399, 0.00967245806245393), (400, 0.00967245806245393), (401, 0.00967245806245393), (402, 0.00967245806245393), (403, 0.00967245

In [13]:

# Transforming dataset values
# vetor de frequencia de palavras = repos freq.
bow_corpus  = [dictionary.doc2bow(text) for text in list(repos)]

# Calcula matriz esparsa de similaridades User_LangsxUser_Langs
index = SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12418)

2018-07-02 22:30:08,546 : INFO : creating sparse index
2018-07-02 22:30:08,546 : INFO : creating sparse matrix from corpus
2018-07-02 22:30:08,548 : INFO : PROGRESS: at document #0
2018-07-02 22:30:08,658 : INFO : created <1992x12418 sparse matrix of type '<class 'numpy.float32'>'
	with 16362 stored elements in Compressed Sparse Row format>


** LISTA DE TOP 10 USUARIOS RECOMENDADOS PARA O USUARIO 4 **

In [42]:
# LISTA DE TOP 10 USUARIOS RECOMENDADOS PARA O USUARIO 4

similarities = index[tfidf[new_vec]]
recomendations = list(enumerate(similarities))
top_recomendations = sorted(recomendations, key=lambda rating: rating[1], reverse=True)
top_users = top_recomendations[:10]
repos_langs.iloc[[tr[0] for tr in top_users], 0]

4           luanfonceca
9        renatooliveira
1653    tisstech-brasil
519             caiofsr
682        ericksantana
66              vanessa
737           betegelse
171          caionovaes
665        MaryCaroline
184           mulatinho
Name: User, dtype: object

end model.